In [133]:
from selenium import webdriver  # 동적크롤링
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [134]:
import time

import warnings
warnings.filterwarnings('ignore')

In [135]:
driver = webdriver.Chrome() 

In [136]:
url = "https://map.kakao.com/"
driver.get(url)

In [137]:
searchloc = '경주'

search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]') # 카카오맵 검색창
search_area.send_keys(searchloc)  # 검색어 전달
driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # 돋보기 클릭

time.sleep(5)

driver.find_element(By.XPATH, r'//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)  # 장소 탭

In [138]:
rv_list = []  # 정보 저장

In [139]:
def rvNamePrint():
    time.sleep(0.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    rv_lists = soup.select('.placelist > .PlaceItem') # 장소 정보를 모두 가져옴
    for i, space in enumerate(rv_lists):
        temp = []
        
        name = space.select('.head_item > .tit_name > .link_name')[0].text
        score = space.select('.rating > .score > em')[0].text
        addr = space.select('.addr > p')[0].text
        
        # 상세정보 탭으로 이동
        driver.find_element(By.XPATH, r'//*[@id="info.search.place.list"]/li['+str(i+1)+']/div[5]/div[4]/a[1]').send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
        


        rev = extract_review()  # 리뷰 추출 함수 실행
        
        
        # 하나의 리스트로 만들어 rv_list에 추가
        temp.append(name)
        temp.append(score)
        temp.append(addr[3:])
        temp.append(rev)
        
        rv_list.append(temp)

In [140]:
#new
def extract_review():
    # 후기 더보기 버튼을 클릭하는 루프
    while True:
        try:
            # 후기 더보기 버튼 클릭
            
            more_button = driver.find_element(By.LINK_TEXT, "후기 더보기")
            if '후기 접기' in more_button.text: # 마지막 후기 더보기 클릭 후 후기 접기를 교차로 반복 클릭하는 현상 발견하여 후기 접기 클릭 시 종료
                break
            
            #time.sleep(2)
            more_button.click()  # 클릭
            time.sleep(2)  # 페이지 로딩 대기
        except Exception as e:
            # 더보기 버튼이 없거나 클릭 실패 시 종료
            print("NEXT PLACE", e)
            break

    # 페이지 소스를 가져와서 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 후기 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
    rev = []
    
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for review in review_lists:
            comment = review.select('.txt_comment > span')[0].text  # 리뷰
            if len(comment) != 0:
                rev.append(comment)
    # 없으면 빈칸 추가
    else:
        rev.append(' ')
        
    # 다시 검색 탭으로 전환
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    
    return rev


In [141]:
page = 1  # 현재 페이지
page2 = 1  # 5개 중 몇번째인지(버튼이 5개씩있어서 6번째가 되면 다시 1로 바꿔줘야함)

# 한 페이지 당 15개의 장소
# 상위 45개 장소만 추출 

for i in range(1, 4):
    try:
        page2 += 1
        print(page, 'page')
        
        # 페이지 버튼 번호(1에서 5 사이 값)
        #if i > 5:
        #    xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i-5)+']'
        #else:
        xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i)+']'
        
        driver.find_element(By.XPATH, xpath).send_keys(Keys.ENTER)  # 페이지 선택
        
        rvNamePrint()  #  정보 크롤링
        
        
        # page2가 5를 넘어가면 다시 1로 바꿔주고 다음 버튼 클릭
        #if page2 > 5:
        #    page2 = 1
        #    driver.find_element(By.XPATH, r'//* [@id="info.search.page.next"]').send_keys(Keys.ENTER)
        
        page += 1
    
    except:
        break
        
print('크롤링 완료')

1 page
NEXT PLACE Message: no such element: Unable to locate element: {"method":"link text","selector":"후기 더보기"}
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DEF79412+29090]
	(No symbol) [0x00007FF6DEEEE239]
	(No symbol) [0x00007FF6DEDAB1DA]
	(No symbol) [0x00007FF6DEDFEFE7]
	(No symbol) [0x00007FF6DEDFF23C]
	(No symbol) [0x00007FF6DEE497C7]
	(No symbol) [0x00007FF6DEE2672F]
	(No symbol) [0x00007FF6DEE465A2]
	(No symbol) [0x00007FF6DEE26493]
	(No symbol) [0x00007FF6DEDF09D1]
	(No symbol) [0x00007FF6DEDF1B31]
	GetHandleVerifier [0x00007FF6DF29871D+3302573]
	GetHandleVerifier [0x00007FF6DF2E4243+3612627]
	GetHandleVerifier [0x00007FF6DF2DA417+3572135]
	GetHandleVerifier [0x00007FF6DF035EB6+801862]
	(No symbol) [0x00007FF6DEEF945F]
	(No symbol) [0x00007FF6DEEF4FB4]
	(No symbol) [0x00007FF6DEEF5140]
	(No symbo

In [142]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_records(rv_list)
df.to_excel(f'{searchloc}.xlsx')